# Machine Learning to Identify Forams species and and whether the shell is cracked or not

## Finnegan Lab
## Data preprocessing

### Setup

In [1]:
# Imports. Feel free to add or remove as necessary 

# !pip install -Uq tensorflow-datasets==4.5.2
# !pip install tf-nightly -q
import pandas as pd
import numpy as np
np.random.seed(2022)
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn
import os, re, math
import keras
import shutil
from pathlib import Path
from PIL import Image
import tensorflow as tf
import tensorflow_datasets as tfds

%load_ext autoreload
%autoreload 2

In [2]:
#@title mount cloud
from google.colab import auth
auth.authenticate_user()
project_id = 'lucid-loader-347021'
!gcloud config set project {project_id}
!gsutil ls

Updated property [core/project].


To take a quick anonymous survey, run:
  $ gcloud survey

gs://paleo-ml/


In [3]:
from google.colab import drive
drive.mount('/content/drive')
dir_drive_code ='drive/MyDrive/MV1012_SBB_images/ML_projects/Code'
assert os.path.isdir(dir_drive_code), 'unable to find code. Please check your drive path above.'

Mounted at /content/drive


In [4]:
!git clone https://github.com/BryanZWu/finnegan-paleo-ml.git
dir_code = 'finnegan-paleo-ml/Code'
!cd {dir_code}
os.chdir(dir_code)

Cloning into 'finnegan-paleo-ml'...
remote: Enumerating objects: 88, done.
remote: Counting objects: 100% (88/88), done.
remote: Compressing objects: 100% (43/43), done.
remote: Total 88 (delta 30), reused 88 (delta 30), pack-reused 0
Unpacking objects: 100% (88/88), done.


In [45]:
# Data stored in drive specified here
dir_base = '/content/drive/MyDrive/MV1012_SBB_images/'
dir_raw_training_images = f'{dir_base}/Box_Core_images'
dir_ml_home = f'{dir_base}/ML_projects'
dir_google_cloud = 'gs://paleo-ml'
dir_processed_data = dir_out_data = f'{dir_google_cloud}/Processed_data/forams'
dir_training_logs = f'{dir_google_cloud}/Training_logs/'
dir_trained_models = f'{dir_ml_home}/Trained_models/'
dir_train_data = f'{dir_processed_data}/train'
dir_val_data = f'{dir_processed_data}/val'
dir_test_data = f'{dir_processed_data}/test'
dir_debug_processed_data = '/content/training_data_debug'
dir_labels_csv = f'{dir_base}/Final_Diversity_CSVs/ML_master2.csv'

In [20]:
#@title import local project files
from common.constants import *
from common.imports import *
from data_processing import data_processor

In [51]:
def git():
    """ Refetch all files """
    !git fetch
    !git checkout origin -- .

## Processing data and storing

Take data from its original form and crop/resize images as necessary. Store images with its corresponding label.

### Run processing

In [30]:
labels_df = data_processor.create_labels_df(dir_labels_csv)
labels_df.head()


Broken            species     age   test    val
sample_name  object_num                                                   
MV1012-BC-12 1.0         unbroken          NOT FORAM  1994.0  False  False
             2.0         unbroken   suggrunda eckisi  1994.0  False  False
             3.0           broken    bulimina exilis  1994.0  False  False
             4.0         unbroken  nonionella stella  1994.0   True  False
             5.0         unbroken          NOT FORAM  1994.0  False   True

In [52]:
#Run data processing
data_processor.run_processing(dir_processed_data, dir_raw_training_images, labels_df)

unable to find the output for processed training data: gs://paleo-ml/Processed_data/forams. Type "YES" to create it and continue.
NO
processing aborted


In [ ]:
#Run data processing

assert os.path.isdir(dir_processed_data), 'unable to find the output for processed training data'

imgs_dir = os.path.join(drive_dir_path, 'Box_Core_images')


for sample_dir in os.listdir(imgs_dir):
    sample_dir_path = os.path.join(imgs_dir, sample_dir)
    sample_name = re.match(r'(.+)_identify', sample_dir).groups()[0]
    # process_sample_dir(sample_dir_path, sample_name, labels_df, data_dir=dev_data_dir, override=False)
    process_sample_dir(sample_dir_path, sample_name, labels_df, data_dir=out_dir, override=False)


    
 

## Create tensorflow dataset and push to google cloud

In [ ]:
#@title constants
image_size = (416, 416)

input_shape = (*image_size, 3)
drive_prefix = '/content/drive/MyDrive/MV1012_SBB_images/ML_projects/'
 

logdir_base = f'{drive_prefix}/Training_logs/'
trained_model_dir_base = f'{drive_prefix}/Trained_models/'

batch_size = 128 * 2
google_cloud_storage = 'gs://paleo-ml'

In [ ]:

# TODO create the respective sets by IMPORTING the image dataset!
os.chdir(tfds_dir)
# print(os.getcwd())
import forams
training_set = tfds.load('forams', split='train')
validation_set = tfds.load('forams', split='val')
testing_set = tfds.load('forams', split='test')

In [ ]:
# tf.data.experimental.save(training_set, f'{google_cloud_storage}/training{batch_size}')
# tf.data.experimental.save(validation_set, f'{google_cloud_storage}/validation{batch_size}')
# tf.data.experimental.save(testing_set, f'{google_cloud_storage}/testing{batch_size}')
tf.data.Dataset.save(training_set, f'{google_cloud_storage}/training{batch_size}')
tf.data.Dataset.save(validation_set, f'{google_cloud_storage}/validation{batch_size}')
tf.data.Dataset.save(testing_set, f'{google_cloud_storage}/testing{batch_size}')